In [812]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
      
#Hier onder staan alle activatie functies en hun afgeleide
def Sigmoid(i):
    return 1/(1 + np.exp(-i))

def ReLu(i):
    return i * (i > 0)

def AfgeleideSigmoid(i):
    return Sigmoid(i) * (1- Sigmoid(i))

def AfgeleideReLu(i):
    return 1 * (i > 0)

def Softmax(i):
    return np.exp(i) / np.sum(np.exp(i), axis=0)

def AfgeleideSoftmax(i):
    return (Softmax(i) * np.identity(Softmax(i).size) - Softmax(i).transpose() @ Softmax(i))

def convert(x, is_y = False):
    if isinstance(x, pd.DataFrame) or isinstance(x, pd.Series):
        x = x.to_numpy()
    
    ls = []
    i = 0
    
    for row in x:
        if not is_y:
            if not isinstance(row[0], np.ndarray):
                ls.append(np.array([row]))
            else:
                ls.append(row)
        else:
            if not isinstance(x[i], np.ndarray):
                ls.append(np.array([x[i]]))
            else:
                ls.append(x[i]) 
        i += 1
    
    ls 
    
    return np.array(ls)
                        

In [813]:
layers = []

#Functie om een laag te initialiseren
def Init_layer(layertype, neurons, previous_layer = False, function = False):
    if layertype == "input": 
        return{
        "output": np.zeros(neurons)
    }
    elif layertype == "hidden":
        return{
        "weights": np.random.normal(0, 1, size=(previous_layer["output"].size, neurons)),
        "bias": np.random.normal(0, 1, neurons),
        "function": function,
        "output": np.zeros(neurons) 
    }
    elif layertype == "output":
        return{
        "weights": np.random.normal(0, 1, size=(previous_layer["output"].size, neurons)),
        "bias": np.random.normal(0, 1, neurons),
        "function": function,
        "output": np.zeros(neurons)       
    }
        
def Add_layer(layertype, neurons, function = False):
    #Als de nieuwe layer de eerste laag is heeft het geen previous_layer en function nodig
    if len(layers) == 0:
        layers.append(Init_layer(layertype, neurons))
    else:
        previous_layer = layers[len(layers) - 1]
        layers.append(Init_layer(layertype, neurons, previous_layer, function))
        
#zelfde seed voor debugging
np.random.seed(1)

In [814]:
def Feedforward(layer, previous_layer):
    #Eerst moeten we de dotproduct van de input en de weights berekenen
    neuronvalues = np.dot(previous_layer["output"], layer["weights"])
    
    #vervolgens tellen we de bias op bij het dotproduct
    neuronvalues = neuronvalues + layer["bias"]
    
    #vervolgens passen we de activatie functie toe op de opsomming
    if layer["function"] == "relu":
        neuronvalues = ReLu(neuronvalues)
    elif layer["function"] == "softmax":
        neuronvalues = Softmax(neuronvalues)
    elif layer["function"] == "sigmoid":
        neuronvalues = Sigmoid(neuronvalues)
    
    #print("neuronvalues", neuronvalues)   
    #zet de output van de laag op de neuronvalues
    layer["output"] = neuronvalues


def BackPropagate(output_error, current_layer, previous_layer):
    
    #check the function used
    function = current_layer["function"]
    
    #afgeleide error berekenen
    if function == "sigmoid":
        input_error = np.dot(AfgeleideSigmoid(current_layer["output"]) * output_error, current_layer["weights"].T)
    elif function == "relu":
        input_error = np.dot(AfgeleideReLu(current_layer["output"]) * output_error, current_layer["weights"].T)
    elif function == "softmax":
        input_error = np.dot(AfgeleideSoftmax(current_layer["output"]) * output_error, current_layer["weights"].T)
    #print("error", input_error)
    
    
    #print("input", previous_layer["output"].T) 
    #print("output error", output_error) 
    #bereken de error van deze laag
    #print(previous_layer["output"].T.shape)
    #print(output_error.shape)
    weights_error = np.dot(previous_layer["output"].T, output_error)
    #print("weights error", weights_error)
    
    
    
    #update weights en biases
    #print("weights", current_layer["weights"])
    current_layer["weights"] -= learning_rate * weights_error
    #print("Updated weights", current_layer["weights"])
    
    #print("bias", current_layer["bias"])
    current_layer["bias"] -= learning_rate * output_error
    #print("Updated bias", current_layer["bias"])
    
    return input_error
   
   
def TrainNetwork(index, X, y):
     
    #Insert the input
    layers[0]["output"] = X[index]
    
    #loop through the layers for feedforwarding
    layers_to_loop = len(layers) - 1

    for i in range(layers_to_loop):
        Feedforward(layers[i + 1], layers[i])

    #print actual output
    actual_output = y[index]
    #print("Actual error", actual_output)
    
    #layers    
    output_layer = layers[len(layers)-1]

    error = 2*(output_layer["output"] - y[index])/y.size 
    
    for i in reversed(range(1, len(layers))):
        error = BackPropagate(error, layers[i], layers[i - 1])  
        


def TrainWithOneDataset(X, y):
    for i in range(len(y)):
        #print("index", i)        
        TrainNetwork(i, X, y)


def Test(X, y, epoch):
    y_pred = []
    
    for index in range(len(y)):
        #Insert the input
        layers[0]["output"] = X[index]
        #print("Input:", layers[0]["output"])
        
        #loop through the layers for feedforwarding
        layers_to_loop = len(layers) - 1
        for i in range(layers_to_loop):
            Feedforward(layers[i + 1], layers[i])
            
        y_pred.append(RoundToBinary(layers[len(layers) - 1]["output"]))
        
    accuracy = accuracy_score(y, y_pred)
    print("Epoch:", epoch, "Accuracy", accuracy)
        
def RoundToBinary(number):
    if number >= 0.5:
        return True
    else:
        return False       

def TrainTest(X, y):
    #reshape de biases zodat we matrix berekeningen kunnen toepassen
    for i in range(1, len(layers)):  
        layers[i]["bias"].shape = (1, len(layers[i]["bias"])) 
    
    for i in range(epochs):
        TrainWithOneDataset(X, y)
        Test(X, y, i)

In [815]:
learning_rate = 0.3
epochs = 10

Add_layer("input", 2)
Add_layer("hidden", 3, "relu")
Add_layer("output", 1, "sigmoid")

X_train = np.array([[[0,0]], [[0,1]], [[1,0]], [[1,1]]])
y_train = np.array([[False], [True], [True], [False]])
TrainTest(X_train, y_train)

#in de derde epoch is er al een accuracy van 100%

Epoch: 0 Accuracy 0.5
Epoch: 1 Accuracy 0.75
Epoch: 2 Accuracy 1.0
Epoch: 3 Accuracy 1.0
Epoch: 4 Accuracy 1.0
Epoch: 5 Accuracy 1.0
Epoch: 6 Accuracy 1.0
Epoch: 7 Accuracy 1.0
Epoch: 8 Accuracy 0.75
Epoch: 9 Accuracy 0.75


In [816]:
#voor deze opdracht wil ik de iris dataset gebruiken die we ook bij de bootcamp hebben gebruikt
#om het op het begin wat makkelijker te maken wil ik maar 2 soorten bloemen gebruiken
from operator import truediv
import seaborn as sns

iris = sns.load_dataset('iris')

#Ik drop voor nu alle rows voor virginica zodat we een makkelijkere output hebben
iris = iris.loc[iris["species"] != "virginica"]

#Split de dataset in input en output
y_train = iris["species"]
X_train = iris[["sepal_length", "sepal_width", "petal_length", "petal_width"]]

#Verander de output voor binaire waardes
y_train.replace("setosa", 0, inplace = True)
y_train.replace("versicolor", 1, inplace = True)

#leeg de layers list
layers = []

learning_rate = 1
epochs = 25

Add_layer("input", 4)
Add_layer("hidden", 6, "relu")
Add_layer("output", 1, "sigmoid")

X_train = np.array(X_train)
y_train = np.array(y_train)

#we moeten de dataset omzetten naar een dataset die we hier voor kunnen gebruiken
X_train = convert(X_train)

TrainTest(X_train, y_train)

Epoch: 0 Accuracy 0.5
Epoch: 1 Accuracy 0.5
Epoch: 2 Accuracy 0.55
Epoch: 3 Accuracy 0.61
Epoch: 4 Accuracy 0.75
Epoch: 5 Accuracy 0.83
Epoch: 6 Accuracy 0.89
Epoch: 7 Accuracy 0.96
Epoch: 8 Accuracy 0.97
Epoch: 9 Accuracy 0.98
Epoch: 10 Accuracy 0.99
Epoch: 11 Accuracy 0.99
Epoch: 12 Accuracy 0.99
Epoch: 13 Accuracy 0.99
Epoch: 14 Accuracy 0.99
Epoch: 15 Accuracy 0.99
Epoch: 16 Accuracy 0.99
Epoch: 17 Accuracy 0.99
Epoch: 18 Accuracy 0.99
Epoch: 19 Accuracy 0.99
Epoch: 20 Accuracy 0.99
Epoch: 21 Accuracy 1.0
Epoch: 22 Accuracy 1.0
Epoch: 23 Accuracy 1.0
Epoch: 24 Accuracy 1.0
